In [1]:
using BenchmarkTools
using MacroTools
using GraphPPL
using Distributions
using Test
 

In [3]:
using Graphs
using MetaGraphsNext
using Test

graph =  MetaGraph(
        Graph(),
        Label = Symbol,
        VertexData = Symbol)

println(nv(graph))

@test_throws MethodError graph["illegal"] = "not a symbol"
println(nv(graph))

0


1


In [8]:
@new_model function sum_all_and_normal(μ, σ, out)
    for i in 1:10
        local x ~ Normal(μ, σ)
    end
    out ~ Normal(μ= x, σ= σ)
end
GraphPPL.plot_graph(sum_all_and_normal())

false
Any[:μ, :σ]


LoadError: LoadError: type Array has no field head
in expression starting at /Users/wnuijten/Documents/biaslab/GraphPPL.jl/src/playground.ipynb:1

In [125]:
@new_model function bigmodel(in_1, in_2, in_3, σ, out)
    μ ~ sum_all_and_normal(; in_1 = in_1, in_2=in_2, in_3=in_3, σ=σ)
    out ~ sum_all_and_normal(; in_1 = μ, in_2=in_2, in_3=in_3, σ=σ)
end
GraphPPL.plot_graph(bigmodel())

In [127]:
@new_model function evenbiggermodel(in_1, in_2, in_3, σ, out)
    μ ~ bigmodel(; in_1 = in_1, in_2=in_2, in_3=in_3, σ=σ)
    out ~ bigmodel(; in_1 = μ, in_2=in_2, in_3=in_3, σ=σ)
end
GraphPPL.plot_graph(GraphPPL.convert_to_ffg(evenbiggermodel()))

In [21]:
eval(GraphPPL.generate_get_or_create_expression(:([in_1 in_2; in_3 in_4])))

GraphPPL.NodeLabel(:in_4, 4)

In [30]:
GraphPPL.interfaces(::typeof(Normal), ::Val{3}) = (:μ, :σ, :out)

@new_model function gcv(x, y, w, γ, θ)
    tmp_1 := prod(w, γ)
    log_σ := sum(tmp_1, θ)
    σ := exp(log_σ)
    y ~ Normal(μ, σ)
end



begin
    function gcv end
    function GraphPPL.make_node!(model, ::GraphPPL.Composite, parent_context, ::typeof(gcv), interfaces)
        context = GraphPPL.Context(parent_context, gcv)
        GraphPPL.copy_markov_blanket_to_child_context(context, interfaces)
        w = if @isdefined(w)
                w
            else
                GraphPPL.getorcreate!(model, context, :w)
            end
        γ = if @isdefined(γ)
                γ
            else
                GraphPPL.getorcreate!(model, context, :γ)
            end
        tmp_1 = if @isdefined(tmp_1)
                tmp_1
            else
                GraphPPL.getorcreate!(model, context, :tmp_1)
            end
        interfaces_tuple = (in = [w, γ], out = tmp_1)
        GraphPPL.make_node!(model, context, prod, interfaces_tuple)
        tmp_1 = if @isdefined(tmp_1)
                tmp_1
            else
                GraphPPL.getorcreate!(model, context, :tmp_1)
            end
        θ = if @isdefined(θ)
  

In [8]:
GraphPPL.plot_graph(GraphPPL.convert_to_ffg(gcv()))

In [15]:
@new_model function chain_of_gcv(w, γ, θ, x, y)
    μ_1 ~ gcv(;x=x, w=w, γ=γ, θ=θ)
    μ_2 ~ gcv(;x=μ_1, w=w, γ=γ, θ=θ)
    μ_3 ~ gcv(;x=μ_2, w=w, γ=γ, θ=θ)
    y ~ gcv(;x=μ_3, w=w, γ=γ, θ=θ)
end

In [19]:
GraphPPL.plot_graph(chain_of_gcv())

In [5]:
using Test

macro return_2(func)
    return func
end

@testset "test" begin
    x = @return_2 function abc()
        return 2
    end
    @test x() == 2
end

Test Summary: | Pass  Total  Time
test          |    1      1  0.0s


Test.DefaultTestSet("test", Any[], 1, false, false, true, 1.67947512088194e9, 1.679475120882044e9)